<a href="https://colab.research.google.com/github/LucianaNieto/CarbonSequestration/blob/main/SOC_ETL_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


* Loading the Data from Excel
* Cleaning and Normalizing Text Data
* Mapping Values
* Standardizing Metrics
* Filtering Data
* Dropping Unnecessary Columns
* Standardizing Country Names
* Creating Directories
* Saving the Processed Data

### Libraries

In [3]:
!pip install -q pycountry fuzzywuzzy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 21.5 MB/s eta 0:00:00


In [42]:
# Import necessary libraries
import pandas as pd
import os
import numpy as np
from pycountry import countries
from fuzzywuzzy import process
import re
import traceback

### Directories

In [43]:

# Define base path and subdirectory paths
base_path = '/content/drive/My Drive/2024/DataScience/CarbonSequestration/01-CarbonSequestration/'
paths = {
    'raw_data_path': base_path + '01-Data/01-RawData/',
    'processed_data_path': base_path + '01-Data/02-ProcessedData/',
    'exploratory_figures_path': base_path + '02-Figures/01-Exploratory/',
    'final_figures_path': base_path + '02-Figures/02-Finals/',
    'raw_code_path': base_path + '03-Code/01-RawCode/',
    'data_processing_code_path': base_path + '03-Code/02-DataProcessing/',
    'data_analysis_code_path': base_path + '03-Code/03-DataAnalysis/',
    'clean_scripts_path': base_path + '03-Code/04-CleanScripts/'
}

### Funtions

In [44]:

# Check and create directory if it does not exist
def check_create_directory(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
            print(f"Directory created: {path}")
        else:
            print(f"Directory already exists: {path}")
    except Exception as e:
        print(f"Failed to check or create directory {path}. Error: {e}")
        traceback.print_exc()

def load_data(sheet_name):
    csv_path = os.path.join(paths['processed_data_path'], f"{sheet_name}.csv")
    return pd.read_csv(csv_path)

def print_unique_values(df, column_list):
    for column in column_list:
        unique_values = df[column].unique()
        print(f"Unique values in {column}: {unique_values}\nTotal unique in {column}: {len(unique_values)}")


def clean_column_names(df):
    """Clean column names by stripping whitespace and converting to lowercase."""
    df.columns = df.columns.str.strip().str.lower()
    return df


def normalize_text(df, column):
    """Normalize text data in a specified column."""
    df[column] = df[column].str.lower().str.strip()
    df[column] = df[column].str.replace(' ; ', ',').str.replace(' -> ', '->')
    df[column] = df[column].str.replace(' : ', ':').str.replace(' , ', ',').str.replace(', ', ',')
    return df


def map_control_values(df, column, mapping):
    """Map control values to standardized categories."""
    df[column] = df[column].map(mapping).fillna(df[column])
    return df

def standardize_metrics(df, metrics):
    """Standardize metrics and filter data."""
    df['metric'] = df['metric'].str.lower()
    return df[df['metric'].isin(metrics)]

def convert_to_ratio(value):
    """Convert percentage to ratio."""
    try:
        return 1 + (float(value) / 100)
    except (ValueError, TypeError):
        return value  # Return original value if conversion fails

def apply_transformations(df, condition, columns):
    """Apply transformations to specified columns under a condition."""
    for column in columns:
        df.loc[condition, column] = df.loc[condition, column].apply(convert_to_ratio)
    return df


def standardize_country_names(country_name):
    if pd.isna(country_name):
        return 'None'
    country_name = country_name.strip().lower()
    # Handling general identifiers like 'Global' or 'All continents'
    if country_name in ['global', 'all continents', 'world', 'worldwide']:
        return 'Global'
    # Splitting entries with multiple countries
    if ',' in country_name:
        country_parts = country_name.split(',')
        standardized_parts = [standardize_single_country(part.strip()) for part in country_parts if part.strip()]
        return ', '.join(standardized_parts)
    # Standardize a single country name
    return standardize_single_country(country_name)

def standardize_single_country(single_country):
    single_country = ''.join(e for e in single_country if e.isalnum() or e.isspace())
    try:
        # If the country name is a direct match
        return countries.lookup(single_country).name
    except LookupError:
        # If no direct match, use fuzzy matching to find the closest valid country name
        possible_matches = [country.name for country in countries]
        closest_match = process.extractOne(single_country, possible_matches, score_cutoff=80)
        if closest_match:
            return closest_match[0]
        return 'None'

def standardize_duration(duration):
    if pd.isna(duration):
        return None
    duration = duration.strip().lower()

    # Convert durations in days to years
    if 'day' in duration:
        days = re.findall(r'\d+', duration)
        if days:
            return float(days[0]) / 365
    # Convert durations in months to years
    elif 'month' in duration:
        months = re.findall(r'\d+', duration)
        if months:
            return float(months[0]) / 12
    # Convert durations in years
    elif 'yr' in duration or 'year' in duration:
        duration = duration.replace('yr', '').replace('year', '').strip()

    # Handle ranges
    if '-' in duration:
        parts = duration.split('-')
        try:
            return float(parts[1])  # Take the higher value
        except ValueError:
            return None
    if '>' in duration:
        try:
            return float(duration.replace('>', '').strip())
        except ValueError:
            return None
    if '<' in duration:
        try:
            return float(duration.replace('<', '').strip())
        except ValueError:
            return None
    try:
        return float(re.findall(r'\d+', duration)[0])
    except (ValueError, IndexError):
        return None


def normalize_depth(value):
    if pd.isna(value):
        return np.nan

    value = value.strip().lower()

    # Handle range
    match = re.match(r'(\d+)\s*-\s*(\d+)\s*cm', value)
    if match:
        _, high = map(int, match.groups())
        return high

    # Handle single values
    match = re.match(r'(\d+)\s*cm', value)
    if match:
        return int(match.group(1))

    # Handle cases like '>30 cm'
    match = re.match(r'>(\d+)\s*cm', value)
    if match:
        return int(match.group(1))

    # For any other non-numeric values, return NaN
    return np.nan




### Data loading

In [46]:
for path in paths.values():
    check_create_directory(path)

Directory already exists: /content/drive/My Drive/2024/DataScience/CarbonSequestration/01-CarbonSequestration/01-Data/01-RawData/
Directory already exists: /content/drive/My Drive/2024/DataScience/CarbonSequestration/01-CarbonSequestration/01-Data/02-ProcessedData/
Directory already exists: /content/drive/My Drive/2024/DataScience/CarbonSequestration/01-CarbonSequestration/02-Figures/01-Exploratory/
Directory already exists: /content/drive/My Drive/2024/DataScience/CarbonSequestration/01-CarbonSequestration/02-Figures/02-Finals/
Directory already exists: /content/drive/My Drive/2024/DataScience/CarbonSequestration/01-CarbonSequestration/03-Code/01-RawCode/
Directory already exists: /content/drive/My Drive/2024/DataScience/CarbonSequestration/01-CarbonSequestration/03-Code/02-DataProcessing/
Directory already exists: /content/drive/My Drive/2024/DataScience/CarbonSequestration/01-CarbonSequestration/03-Code/03-DataAnalysis/
Directory already exists: /content/drive/My Drive/2024/DataScie

In [47]:
#load data and read sheets
excel_file_name = 'SOC_db.xlsx'
excel_file_path = os.path.join(paths['raw_data_path'], excel_file_name)
try:
    xls = pd.ExcelFile(excel_file_path)
    print(xls.sheet_names)
except Exception as e:
    print(f"An error occurred while loading the Excel file: {e}")

# Dictionary to hold dataframes for each sheet
sheets_df = {}

for sheet_name in xls.sheet_names:
    sheets_df[sheet_name] = pd.read_excel(xls, sheet_name=sheet_name)
    print(f"////////////////First few rows of {sheet_name}:")
    print(sheets_df[sheet_name].head())

    # stat
    print(f"////////////////Info for {sheet_name}:")
    sheets_df[sheet_name].info()
    print(f"////////////////Description for {sheet_name}:")
    print(sheets_df[sheet_name].describe())

['readme', 'header_names', 'retained_meta-analyses', 'non_retained_meta-analyses', 'Effect-sizes', 'Primary_studies']
////////////////First few rows of readme:
           Unnamed: 0  Unnamed: 1  \
0                 NaN         NaN   
1  Global information         NaN   
2                 NaN         NaN   
3                 NaN         NaN   
4                 NaN         NaN   

                                          Unnamed: 2  
0                                                NaN  
1                      Data associated to the paper:  
2                                                NaN  
3  A global database of land management, land-use...  
4  Authors: Damien Beillouin1,2*, Julien Demenois...  
////////////////Info for readme:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  5 non-null      object 
 1   Unnamed: 1  0 non-null  

In [48]:
# sheet names to keep and save as files
sheets_to_keep = ['retained_meta-analyses', 'Effect-sizes', 'Primary_studies']

for sheet_name in xls.sheet_names:
    if sheet_name in sheets_to_keep:
        sheets_df[sheet_name] = pd.read_excel(xls, sheet_name=sheet_name)
        csv_path = os.path.join(paths['processed_data_path'], f"{sheet_name}.csv")
        sheets_df[sheet_name].to_csv(csv_path, index=False)
        print(f"{sheet_name} saved to {csv_path}")


retained_meta-analyses saved to /content/drive/My Drive/2024/DataScience/CarbonSequestration/01-CarbonSequestration/01-Data/02-ProcessedData/retained_meta-analyses.csv
Effect-sizes saved to /content/drive/My Drive/2024/DataScience/CarbonSequestration/01-CarbonSequestration/01-Data/02-ProcessedData/Effect-sizes.csv
Primary_studies saved to /content/drive/My Drive/2024/DataScience/CarbonSequestration/01-CarbonSequestration/01-Data/02-ProcessedData/Primary_studies.csv


### ETL

In [49]:
#files per sheet
df_retained = load_data('retained_meta-analyses')
print(df_retained.columns)
print(df_retained.shape)

df_effects = load_data('Effect-sizes')
print(df_effects.columns)
print(df_effects.shape)


df_primary = load_data('Primary_studies')
print(df_primary.columns)
print(df_primary.shape)

Index(['ID', 'SOC_as_covariable', 'Quality_Unit',
       'Quality_appraisal of primary studies', 'Quality_Lit_gray_and_peer',
       'Quality_Lit_Nb_database', 'Quality_Lit_strings_OK',
       'Quality_Lit_inclusion_exclusion', 'Quality_Model',
       'Quality_Model_weights', 'Quality_Model_heterogeneity (I2, PI, …)',
       'Quality_Publication_bias', 'soil_depth', 'length_time_series',
       'Quality_Data_Sharing', 'Population: geographical scale',
       'Field experiment', 'Lab, incubation, greenhouse', 'Other',
       'Mitigation', 'Adaptation', 'FoodSecurity', 'Source', 'Article Title',
       'Abstract', 'Keywords', 'Authors', 'Year', 'Journal', 'link',
       'Keywords Plus', 'Addresses', 'Email Addresses', 'Funding Orgs',
       'Cited Reference Count', 'Times Cited, WoS Core',
       'Times Cited, All Databases', 'WoS Categories', 'Research Areas',
       'reviewer'],
      dtype='object')
(217, 40)
Index(['NUM', 'ID', 'Figure', 'Land_use', 'Intervention',
       'Sub_Cat_in

In [50]:
# Columns to drop from each dataframe
columns_to_drop_df_retained = [
    'Quality_appraisal of primary studies', 'Quality_Lit_gray_and_peer',
    'Quality_Lit_Nb_database', 'Quality_Lit_strings_OK',
    'Quality_Lit_inclusion_exclusion', 'Quality_Model',
    'Quality_Model_weights', 'Quality_Model_heterogeneity (I2, PI, …)',
    'Lab, incubation, greenhouse', 'Other',
    'Mitigation', 'Adaptation', 'FoodSecurity', 'Source', 'Article Title',
    'Abstract', 'Keywords', 'Authors', 'Year', 'Journal', 'link',
    'Keywords Plus', 'Addresses', 'Email Addresses', 'Funding Orgs',
    'Cited Reference Count', 'Times Cited, WoS Core',
    'Times Cited, All Databases', 'WoS Categories', 'Research Areas',
    'reviewer'
]

columns_to_drop_df_effects = [
    'NUM', 'Figure', 'Inverse_ES', 'Homogenized_response', 'comments', 'Reviewer'
]

columns_to_drop_df_primary = [
    'Primary studies_name', 'Primarystudies_Region', 'Article Title',
    'Authors', 'Author Full Names', 'Abstract', 'Source Title', 'Language',
    'Document Type', 'Author Keywords', 'Addresses', 'Cited References',
    'ISSN', 'Journal Abbreviation', 'Journal ISO Abbreviation',
    'Publication Year', 'WoS Categories', 'Research Areas', 'Reviewer'
]


df_retained.drop(columns=columns_to_drop_df_retained, inplace=True)
print("Updated df_retained columns:", df_retained.info())

df_effects.drop(columns=columns_to_drop_df_effects, inplace=True)
print("Updated df_effects columns:", df_effects.info())

df_primary.drop(columns=columns_to_drop_df_primary, inplace=True)
print("Updated df_primary columns:", df_primary.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 9 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   ID                              217 non-null    object
 1   SOC_as_covariable               217 non-null    object
 2   Quality_Unit                    199 non-null    object
 3   Quality_Publication_bias        217 non-null    object
 4   soil_depth                      217 non-null    object
 5   length_time_series              215 non-null    object
 6   Quality_Data_Sharing            217 non-null    object
 7   Population: geographical scale  201 non-null    object
 8   Field experiment                217 non-null    object
dtypes: object(9)
memory usage: 15.4+ KB
Updated df_retained columns: None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15857 entries, 0 to 15856
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
--

In [51]:
# Columns for which to find unique elements
unique_columns_df_retained = ['SOC_as_covariable', 'Quality_Unit',
       'soil_depth', 'length_time_series',
        'Field experiment']
unique_columns_df_effects = ['Land_use', 'Intervention', 'Sub_Cat_intervention', 'details',
       'Control (C)', 'treatment_authors', 'management',
       'method', 'species', 'region_climate', 'soil', 'depth_original',
       'depth', 'group_depth', 'depth2', 'Duration', 'carbon', 'unit',
       'Rateyr', 'Log_scale', 'metric', 'Outcome', 'Sub_cat_outcome',
       'details_outcome',
       'N_paired_data']
unique_columns_df_primary = ['Primarystudies_Country', 'intervention', 'Outcome']

print("//////////////////Unique values for df_retained:")
print_unique_values(df_retained, unique_columns_df_retained)

print("//////////////////Unique values for df_effects:")
print_unique_values(df_effects, unique_columns_df_effects)

print("//////////////////Unique values for df_primary:")
print_unique_values(df_primary, unique_columns_df_primary)

//////////////////Unique values for df_retained:
Unique values in SOC_as_covariable: ['YES' 'NO']
Total unique in SOC_as_covariable: 2
Unique values in Quality_Unit: [nan 'YES, MM' 'NO' 'YES, Results' 'Yes, MM' 'YES, supMat' 'YES, others']
Total unique in Quality_Unit: 7
Unique values in soil_depth: ['not analysed/detailed' 'yes, subgroups according to soil depth'
 'mixed different layers/all regrouped' 'yes, regression'
 'aggregated/normalized/restricted to 0-15 cm'
 'aggregated/normalized/restricted to 3-100 cm'
 'aggregated/normalized/restricted to 7-30 cm'
 'aggregated/normalized/restricted to 0-20 cm'
 'aggregated/normalized/restricted to 0-30 cm' 'all > 30cm'
 'aggregated/normalized/restricted to 0-100 cm'
 'aggregated/normalized/restricted to 15-30 cm' 'considered in the model'
 'yes, up to > 100 cm' 'only the uppermost layer' 'yes, up to > 30 cm'
 'aggregated/normalized/restricted to 0-50 cm' 'averaged depth'
 'yes, up to 40cm']
Total unique in soil_depth: 19
Unique values in l

In [52]:
#Country names

df_primary['Primarystudies_Country'] = df_primary['Primarystudies_Country'].apply(standardize_country_names)

print(df_primary['Primarystudies_Country'].unique())

#save processed primary studies after fixing country name
filename01 = 'df_Primarystudies_Country.csv'
full_path = os.path.join(paths['processed_data_path'], filename01)
df_primary.to_csv(full_path, index=False)

print(f'DataFrame successfully saved to: {full_path}')
print(f"Total entries in the processed dataset: {len(df_primary)}")

['Denmark' 'Sweden' 'Iran, Islamic Republic of' 'Cameroon' 'China'
 'Costa Rica' 'United States' 'Ecuador' 'India' 'Germany' 'None' 'Ukraine'
 'Kenya' 'Malawi' 'Burkina Faso' 'Senegal' 'Taiwan, Province of China'
 'Philippines' 'Australia' 'Japan' 'Indonesia' 'Brazil' 'France' 'Spain'
 'Argentina' 'United Arab Emirates' 'Canada' 'Italy' 'Mali' 'Ghana'
 'Puerto Rico' 'Nepal' 'Tunisia' 'Zambia' 'Mexico' 'Belgium' 'Netherlands'
 'Switzerland' 'Ireland' 'Thailand' 'Ethiopia' 'Serbia' 'Peru' 'Zimbabwe'
 'Global' 'Kenya, Indonesia' 'Estonia' 'Costa Rica, Canada' 'Colombia'
 'South Africa' 'Norway' 'Nigeria' 'Botswana' 'Namibia' 'Finland' 'Chile'
 'New Zealand' 'Uruguay' 'Malaysia' 'Iceland' 'Poland' 'Congo'
 'Uzbekistan' 'Russian Federation' 'Bolivia, Plurinational State of'
 'Guinea-Bissau' 'Panama' 'Czechia' 'Korea, Republic of' 'Benin'
 'Honduras' 'Togo' 'Sudan' 'Viet Nam' 'Portugal' 'Papua New Guinea'
 'Austria' 'Tanzania, United Republic of'
 'Venezuela, Bolivarian Republic of' 'Hungary

In [53]:
#dictionaries with the corrected / reclassified elements
control_mapping = {
    'no biochar': 'no biochar',
    'no fire': 'no fire',
    'no cover crops': 'no cover crops',
    'farmland or abandonned land': 'abandoned land',
    'other land': 'other land',
    'cropland (previous land use)': 'cropland',
    'forest (previous land use)': 'forest',
    'grassland (previous land use)': 'grassland',
    'plantation (previous land use)': 'forest',
    'non fertilized control': 'no fertilization',
    'fertilized control': 'fertilization',
    'unfertilized cntrol': 'no fertilization',
    'pasture/grassland (previous land use)': 'grassland',
    'all previous land use': 'all land use',
    'agriculture (previous land use)': 'cropland',
    'uncultivated/other (previous land use)': 'abandoned land',
    'forest plantation (previous land use)': 'forest',
    'no fire, no harvest': 'no fire, no harvest',
    'no harvest': 'no harvest',
    'urea': 'fertilization, N',
    'other grazing practices': 'grassland',
    'no amendments': 'no amendments',
    'no residues amendments': 'no amendments',
    'various land': 'various land use',
    'no ferti': 'no fertilization',
    'synthetic fertilizer only': 'fertilization',
    'without fresh c input': 'no carbon added',
    'grassland': 'grassland',
    'no fertilizer added': 'no fertilization',
    'simplified rotation': 'rotation',
    'mineral only fertilization': 'fertilization',
    'no organic material added': 'no fertilization',
    'no organic amendments': 'no amendments',
    'inorganic fertilizer': 'fertilization',
    'no fertilization': 'no fertilization',
    'mineral fertilization': 'fertilization',
    'conventional management (no c input)': 'conventional',
    'conventional management': 'conventional',
    'synthetic n': 'fertilization, N',
    'straw removal': 'no straw',
    'no straw retention': 'no straw',
    'no straw return': 'no straw',
    'no n fertilization': 'no fertilization',
    'organic amendment': 'amendment',
    'no input at all': 'no fertilization',
    'n fertilizer': 'fertilization, N',
    'conventional management (synthetic fertilization, conventional tillage and non-use of organic inputs, except crop residues in some cases)': 'conventional',
    'no grazing': 'no grazing',
    'no grazing exclusion': 'no grazing',
    'continuous grazing': 'grassland',
    'no drought': 'no drought',
    'no irrigation': 'no irrigation',
    'non freeze-thaw': 'no freeze-thaw',
    'no watering': 'no irrigation',
    'no warming': 'no warming',
    'no wetting effect': 'no irrigation',
    'no co2 enrichment': 'no CO2',
    'no elevated c02': 'no CO2',
    'all': 'all',
    'no n added': 'no fertilization',
    'non-warming': 'no warming',
    'no snow manipulation': 'no snow removal',
    'no rainfall reduction': 'no drought',
    'no c02 elevation and non ferti': 'no CO2',
    'no c02 elevation': 'no CO2',
    'no n ferti': 'no fertilization',
    'before-after': 'before and after',
    'previous land-use': 'other land use',
    'cropland': 'cropland',
    'degraded fallow': 'fallow',
    'unshaded system': 'unshaded',
    'underutilised land': 'underuse',
    'fallow land': 'fallow',
    'fallow': 'fallow',
    'unshaded': 'unshaded',
    'underutilized': 'underuse',
    'sole cropping, continuous cropping without trees, and plots outside tree crowns in the case of parklands': 'cropland',
    'tillage: conventional': 'conventional',
    'pasture': 'grassland',
    'forest or agricultural land use or pasture': 'various land use',
    'agriculture land': 'cropland',
    'forest': 'forest',
    'uncultivated land': 'underuse',
    'agrisilviculture': 'agrosilviculture',
    'cropland/forest land/grassland to trop_agrosilvopasture': 'various land use',
    'forest or agricultural land use or pasture or unultivated': 'various land use',
    'grassland/other land to trop_silvopasture': 'grassland',
    'forest or agricultural land use': 'cropland and forest',
    'grassland_other land to trop_agrisilvopasture': 'grassland',
    'cropland->multistrata systems': 'cropland',
    'cropland/forest to temp_agrisilviculture': 'cropland and forest',
    'forest/pasture to temp_silvopasture': 'forest and grassland',
    'other land to temp_agrosilvopasture': 'other land use',
    'cropland/grassland to arid_agrisilviculture': 'cropland and grassland',
    'cropland/grassland/other land to trop_agrisilviculture': 'cropland and grassland',
    'cropland/forest to trop_agrisilviculture': 'cropland and forest',
    'forest land/grassland to trop_agrosilvopasture': 'forest and grassland',
    'cropland/forest land/grassland to silvopasture': 'cropland and forest',
    'cropland/forest land/grassland/other lands to agrisilviculture': 'various land use',
    'forest land/grassland to silvopasture': 'forest and grassland',
    'bare fallow': 'fallow',
    'no temperature increase': 'no warming',
    'no precipitation increase': 'no irrigation',
    'no precipitation reduction': 'no drought',
    'no np fertilization': 'no fertilization',
    'no p fertilization': 'no fertilization',
    'monoculture': 'monoculture',
    'rotation with lower functional richness': 'rotation',
    'non agroforestry': 'no agroforestry',
    'cropland or pasture': 'cropland',
    'no hedgerow': 'no hedgerow',
    'no grass strips': 'no grassland',
    'conventional agriculture': 'conventional',
    'clean tillage': 'conventional',
    'non n addition': 'no fertilization',
    'no n addition': 'no fertilization',
    'no ash application': 'no amendments',
    'unfertilized': 'no fertilization',
    'without earthworms': 'no earthworms',
    'no p': 'no fertilization',
    'no se fertilization': 'no fertilization',
    'organic fertilizer (initial stage)': 'fertilization',
    'combined chemical + organic fertilizer (initial stage)': 'fertilization',
    'no fertilizer (initial stage)': 'no fertilization',
    'chemical fertilizer (initial stage)': 'fertilization',
    'no straw return (initial stage)': 'no straw',
    'no retention': 'no straw',
    'no manure noferti': 'no fertilization',
    'no manure': 'no fertilization',
    'no manure, no ferti': 'no fertilization',
    '100% inorganic n addition': 'fertilization, N',
    'no inorganic n addition': 'no fertilization',
    'one shot inorganic n addition': 'fertilization, N',
    'top placement of inorganic n addition': 'fertilization, N',
    'normal inorganic n addition': 'fertilization, N',
    'no liming': 'no fertilization',
    'no p addition': 'no fertilization',
    'no fertilizer, liming and grazing': 'no fertilization',
    'continuous flooding': 'flooding',
    'young ex arable': 'cropland',
    'arable': 'cropland',
    'ancient grassland': 'grassland',
    'conventional till': 'conventional',
    'non afforested': 'no forest',
    'agricultural': 'cropland',
    'agriculture cultivated': 'cropland',
    'agriculture unknown': 'cropland',
    'all land use': 'various land use',
    'agriculture forage': 'cropland',
    'wildland': 'various land use',
    'agriculture (cropland + pasture) (previous land use)': 'cropland',
    'pasture (previous land use)': 'grassland',
    'natural grassland (previous land use)': 'grassland',
    'former land use - cropland': 'cropland',
    'no afforestation': 'no forest',
    'former land use - grassland': 'grassland',
    'former land use - heathland': 'heathland',
    'former land use - barrenland': 'barrenland',
    'natural grassland': 'grassland',
    'no afforestation (previous land use)': 'no forest',
    'non forested land': 'no forest',
    'initial soc stock': 'no fertilization',
    'cropland/pasture/bare area': 'cropland and grassland',
    'bare area': 'bare soil',
    'medium c input': 'fertilization',
    'conventional practices': 'conventional',
    'native': 'native',
    'other land use': 'other land use',
    'cultivated land': 'cropland',
    'wood land': 'shrubland',
    'natural vegetation': 'native',
    'primary forest': 'forest',
    'secondary forest': 'forest',
    'forest all': 'forest',
    'no forest harvesting': 'forest',
    'grassland or pasture': 'grassland',
    'barren land': 'barrenland',
    'primary or secondary forest': 'forest',
    'grassland and cropland': 'cropland and grassland',
    'continuous maize': 'monoculture',
    'no fertilizer': 'no fertilization',
    'no-tillage': 'no till',
    'undisturbed mangrove (previous land use)': 'native',
    'crop': 'cropland',
    'overall': 'all',
    'crop (previous land use)': 'cropland',
    'savanna (previous land use)': 'savanna',
    'farmland (previous land use)': 'cropland',
    'all previous land uses': 'various land use',
    'primary forest (previous land use)': 'forest',
    'secondary forest (previous land use)': 'forest',
    'arable land': 'cropland',
    'forest land': 'forest',
    'perennial plantation (previous land use)': 'forest',
    'native forest (previous land use)': 'native',
    'long fallow swidden': 'fallow',
    'uncut control': 'native',
    'natural forest': 'native',
    'grazed grassland (previous land use)': 'grassland',
    'pure stand norway spruce': 'fallow',
    'shrub encroachment': 'shrubland',
    'non-crop land': 'forest',
    'plantation': 'forest',
    'wetland': 'wetland',
    'control wetland within 10km, or before-after': 'wetland',
    'previous land: forest': 'forest',
    'grass': 'grassland',
    'previous land: grass': 'grassland',
    'natural wetland': 'wetland',
    'woody plants retained': 'shrubland',
    'before biological intervention': 'non explicit',
    'before exclosure': 'non explicit',
    'before bunds': 'non explicit',
    'before bunds and biological intervention': 'non explicit',
    'before terrassing': 'non explicit',
    'before conservation agriculture': 'non explicit',
    'no improved management practice': 'non explicit',
    'no organic manure': 'no fertilization',
    'mineral fertilizer': 'fertilization',
    'conventional tillage': 'conventional',
    'no organic matter input': 'no fertilization',
    'no co2 change': 'no CO2',
    'no fetilizer': 'no CO2',
    'no till': 'no till',
    'residue removal': 'no straw',
    'residue removed': 'no straw',
    'high intensity tillage (e.g. mouldboard plough and subsoiling)': 'conventional',
    'intermediate tillage (e.g. chisel, disk, harrow and mulch plough)': 'conventional',
    'chemical fertilization application': 'fertilization',
    'plow tillage with residue removal': 'conventional',
    'no litter removal': 'no till',
    'no litter addition': 'no litter',
    'no litter exclusion': 'litter',
    'no litter excluision': 'litter',
    'plow tillage + residue removed': 'conventional',
    'plow tillage + residue retention': 'conventional',
    'no-tillage (previous land-use)': 'no till',
    'savannah/forest': 'savanna',
    'savannah': 'savanna',
    'full tillage': 'conventional',
    'conventional farming': 'conventional',
    'no plastic film': 'no cover',
    'no organic c addition': 'no fertilization',
    'before after': 'before and after',
    'non removal': 'no till',
    '-': 'none',
    '1' : 'none',
    'crop frequency: single (1 crop per year)': 'cropland',
    'high intensity tillage': 'conventional',
    'intermediate intensity tillage': 'conventional',
    'conventional tillage without residues': 'conventional',
    'tillage': 'conventional',
    'conventionnal tillage': 'conventional',
    'bare soil': 'bare soil',
    'no amendemenrts': 'no amendments',
    'traditional tillage + no cover crop': 'conventional',
    'traditional tillage': 'conventional',
    'no high biomass rice cultivar': 'low biomass',
    'non polluted soils': 'no polluted',
    'no pollution': 'no polluted',
    'non shrub-encroached grassland': 'grassland',
    'no acid addition': 'no amendments',
    'intercanopy microsite': 'intercanopy',
    'no biotic disturbance': 'native',
    'no co2, no warming': 'no CO2, no warming',
    'no co2': 'no CO2',
    'nondegraded alpine meadow': 'alpine',
    'conventional forest harvest (tree stems)': 'forest',
    'unharvested forests': 'forest',
    'natural succession': 'native',
    'non harvested + chronosequence': 'no harvest',
    'no c02 + drought': 'no CO2, no irrigation',
    'no co2 + drought': 'no CO2, no irrigation',
    'no co2 + rainfall': 'no CO2, irrigation',
    'no co2+n': 'no CO2, fertilization, N',
    'no co2 + n': 'no CO2, fertilization, N',
    'no c02+n': 'no CO2, fertilization, N',
    'no co2 + warming': 'no CO2, Warming',
    'no co2+ warming': 'no CO2, Warming',
    'no c02 + warming': 'no CO2, Warming',
    'no c02+ warming': 'no CO2, Warming',
    'no n + drought': 'no N, no irrigation',
    'no n+ rainfall': 'no N, irrigation',
    'no n+ p addition': 'no N, yes P',
    'no n +rainfall': 'no N, irrigation',
    'no warming + drought': 'no warming, no irrigation',
    'no warming + rainfall': 'no warming, irrigation',
    'no increase in rainfall': 'no increase in rainfall',
    'no warming + n': 'no warming, fertilization, N',
    'no warming +n': 'no warming, fertilization, N',
    'no rainfall': 'no irrigation',
    'no n': 'no N',
    'no n+p addition': 'no N, no P',
    'non plant invasion': 'controlled vegetation',
    'non overgrazing': 'no grassland',
    'whole tree harvest': 'forest harvest',
    'no thinning': 'no controlled vegetation',
    'conifer stands': 'forest',
    'non treated urea': 'fertilization, N',
    'degraded grassland': 'degraded grassland',
    'urea (> 200 kg n ha-1)': 'fertilization, N',
    'normal n fertilizer': 'fertilization, N',
    'no straw': 'no Straw',
    'npk fertilizer': 'fertilization, N',
    'rice only': 'monoculture',
    'n fertilizer (294 kg n ha-1)': 'fertilization, N',
    'n fertilizer (320 kg n ha-1)': 'fertilization, N',
    'n fertilizer (264 kg n ha-1)': 'fertilization, N',
    'n fertilizer (268 kg n ha-1)': 'fertilization, N',
    'grazing': 'grassland',
    'heavy grazing': 'grassland',
    'without crop mixture': 'no crop mixture',
    'no intercropping': 'no intercropping',
    'no earthworms or reduced earthworm abundance': 'no earthworms',
    'monoculture plots': 'monoculture'
}


land_use_mapping = {
    'heatland': 'heathland',
    'various land use': 'various',
    'other land : various': 'various',
    'other land': 'various'
}

intervention_mapping = {
    'global changes/management': 'global changes, management',
    'land use change/management': 'land use change, management'
}


In [54]:
df_effects.columns

Index(['ID', 'Land_use', 'Intervention', 'Sub_Cat_intervention', 'details',
       'Control (C)', 'treatment_authors', 'summary_effect_size', 'management',
       'method', 'species', 'region_climate', 'soil', 'depth_original',
       'depth', 'group_depth', 'depth2', 'Duration', 'carbon', 'unit',
       'Rateyr', 'Log_scale', 'metric', 'Outcome', 'Sub_cat_outcome',
       'details_outcome', 'lower_CI', 'Effect size', 'upper_CI', 'p_value',
       'N_paired_data', 'ES_SE'],
      dtype='object')

In [55]:
# Clean column names
df_effects = clean_column_names(df_effects)

# Normalize relevant text columns
text_columns = ['land_use', 'intervention', 'sub_cat_intervention', 'details',
       'control (c)', 'treatment_authors', 'summary_effect_size', 'management',
       'method', 'species', 'region_climate', 'soil', 'carbon', 'unit',
       'rateyr', 'log_scale', 'metric', 'outcome', 'sub_cat_outcome',
       'details_outcome', 'lower_ci', 'effect size', 'upper_ci', 'p_value',
       'n_paired_data', 'es_se',]
for col in text_columns:
    df_effects = normalize_text(df_effects, col)

In [56]:
df_effects.head()

,id,land_use,intervention,sub_cat_intervention,details,control (c),treatment_authors,summary_effect_size,management,method,...,metric,outcome,sub_cat_outcome,details_outcome,lower_ci,effect size,upper_ci,p_value,n_paired_data,es_se
0,"07_Sagrilo et al, 2015",cropland,management,amendments pyrogenic,amendments:biochar,no biochar,biochar,NaN,pyc:soc ratio <2,-,...,ratio,ghg,co2,emissions,0.932926829268292,1.03353658536585,1.14329268292682,NaN,179,NaN
1,"07_Sagrilo et al, 2015",cropland,management,amendments pyrogenic,amendments:biochar,no biochar,biochar,NaN,overall pyc:soc ratio,-,...,ratio,ghg,co2,emissions,1.14329268292682,1.27134146341463,1.43597560975609,NaN,268,NaN
2,"07_Sagrilo et al, 2015",cropland,management,amendments pyrogenic,amendments:biochar,no biochar,biochar,NaN,pyc:soc ratio >2,-,...,ratio,ghg,co2,emissions,1.55487804878048,1.98475609756097,2.63871951219512,NaN,89,NaN
3,"07_Sagrilo et al, 2015",cropland,management,amendments pyrogenic,amendments:biochar,no biochar,biochar,NaN,pyc:soc ratio >2,-,...,ratio,ghg,co2,emissions,1.57,2.55,4.53,NaN,23,NaN
4,"07_Sagrilo et al, 2015",cropland,management,amendments pyrogenic,amendments:biochar,no biochar,biochar,NaN,pyc:soc ratio <2,-,...,ratio,ghg,co2,emissions,1.05,1.16,1.27,NaN,29,NaN


In [57]:
#mapping and replacements

df_effects['land_use'] = df_effects['land_use'].replace(land_use_mapping)

df_effects['intervention'] = df_effects['intervention'].replace(intervention_mapping)

#control (C) mapping
df_effects = map_control_values(df_effects, 'control (c)', control_mapping)

In [58]:
df_effects['duration_standardized'] = df_effects['duration'].apply(standardize_duration)

df_effects['depth'] = df_effects['depth'].apply(normalize_depth)

In [59]:
# Standardize metrics and filter data
metrics_to_keep = ['ratio', 'percent change', "hedge's d", 'standardized difference']
df_effects_filtered = standardize_metrics(df_effects, metrics_to_keep)

# Filter for 'soil carbon'
df_effects_filtered = df_effects_filtered[df_effects_filtered['outcome'].str.lower() == 'soil carbon']

# Convert percent to ratio where applicable
condition = df_effects_filtered['metric'] == 'percent change'
columns_to_transform = ['lower_ci', 'effect size', 'upper_ci']
df_effects_filtered = apply_transformations(df_effects_filtered, condition, columns_to_transform)

In [60]:
#Save the filtered DataFrame
full_path = os.path.join(paths['processed_data_path'], 'df_effects_SOC.csv')
df_effects_filtered.to_csv(full_path, index=False)

print(f'DataFrame successfully saved to: {full_path}')
print(f'Total entries in the processed dataset: {len(df_effects_filtered)}')

DataFrame successfully saved to: /content/drive/My Drive/2024/DataScience/CarbonSequestration/01-CarbonSequestration/01-Data/02-ProcessedData/df_effects_SOC.csv
Total entries in the processed dataset: 5172
